# 1. Install all the required dependencies to run the notebook in Google Colab

## 1.1. Install TensorFlow in Google Colab
Google Colab does not support TensorFlow 1.x any more so we need to install it automatically. After running this section, wait to the sesion to be restarted.

In [ ]:
# Install the correct TF version for Google Colab
!pip uninstall -y -q tensorflow
!pip install -q tensorflow-probability==0.8
!pip install -q kapre==0.1.7
!pip install -q tensorflow==1.15
!pip install imagecodecs
#Force session restart
exit(0)

     |████████████████████████████████| 2.5 MB 4.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gym 0.17.3 requires cloudpickle<1.7.0,>=1.2.0, but you have cloudpickle 1.1.1 which is incompatible.
     |████████████████████████████████| 511.7 MB 5.9 kB/s 
     |████████████████████████████████| 412.3 MB 24 kB/s 
     |████████████████████████████████| 503 kB 73.7 MB/s 
     |████████████████████████████████| 50 kB 6.8 MB/s 
     |████████████████████████████████| 3.8 MB 30.3 MB/s 


## 1.2 Install all the required dependencies for DMNet and the conda environment

In [ ]:
# Clone the GitHub repository
!git clone https://github.com/CIVA-Lab/DMNet_Rina.git

# Install anaconda dependencies to create an environment
!pip install -q condacolab
import condacolab
condacolab.install()

Cloning into 'DMNet_Rina'...
remote: Enumerating objects: 454, done.
remote: Counting objects: 100% (454/454), done.
remote: Compressing objects: 100% (255/255), done.
remote: Total 454 (delta 251), reused 366 (delta 192), pack-reused 0
Receiving objects: 100% (454/454), 1.34 MiB | 8.59 MiB/s, done.
Resolving deltas: 100% (251/251), done.
⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:22
🔁 Restarting kernel...


In [7]:
# Create the conda environment
import os
os.chdir("/content/DMNet_Rina/training_codes")
!conda env create -f environment.yml

# Activate the environment
!source activate cell && conda env list

## Check that TF 1 is being used
import tensorflow as tf
print(tf.__version__)


CondaValueError: prefix already exists: /usr/local/envs/cell

# conda environments:
#
base                     /usr/local
cell                  *  /usr/local/envs/cell

1.15.0


## 1.3 Mount your Google Drive if you want to access your data

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 2. Place pretrained models and the data in the correct folder (optional if it is not done yet)

In [ ]:
# Make a folder for the models
!mkdir "/content/DMNet_Rina/training_codes/models_imagenet"

# Copy the pretrained models in the models folder. Please change the paths with your own paths (Optional, you can place it in a different way)
!cp "/content/drive/MyDrive/XXXXXXXXX/hrnetv2_w32_imagenet_pretrained.pth" "/content/DMNet_Rina/training_codes/models_imagenet/hrnetv2_w32_imagenet_pretrained.pth"

# Make a folder for the data
!mkdir "/content/DMNet_Rina/Data"
!mkdir "/content/DMNet_Rina/Data/train"

# Place the data in the data folder (optional, you can place it in a different way)
!cp -R '/content/drive/MyDrive/XXXXXXXXX/datasets/raw/DIC-C2DH-HeLa' '/content/DMNet_Rina/training_codes/Data/train'
!cp -R '/content/drive/MyDrive/XXXXXXXXX/DIC-C2DH-HeLa/01' '/content/DMNet_Rina/training_codes/Data/train/DIC-C2DH-HeLa'


# 3. Run training
Once this cell is executed, part of the GPU memory is occupied and may give memory errors. Restart the session and run the cells again.

In [ ]:
import multiprocessing
#import pandas as pd
import numpy as np
import skimage
#import gdal
import sys
import os
sys.path.insert(0, '/content/DMNet_Rina/training_codes/solaris_rina')
print (sys.path)
sol = __import__('solaris_rina')
os.chdir("/content/DMNet_Rina/training_codes/generate_bash")


# Dataset location (edit as needed)
dataset_namein = "DIC-C2DH-HeLa"
gttype="GT"
branch="mask"
config_path="/content/DMNet_Rina/training_codes/yml/all_eachcell.yml"
# Load config
config = sol.utils.config.parse(config_path)


# %% ============================
# Training
# ===============================

# make model output dir



os.makedirs(os.path.dirname(config['training']['model_dest_path']), exist_ok=True)
config['pretrained_rina']=True
# Adapt the batch size
config['batch_size'] = 1
config['training']['epochs']=10

trainer = sol.nets.train_cell_GTST.Trainer(config=config,dataset_name=dataset_namein,branch=branch,GT=gttype)

trainer.train()

['/content/DMNet_Rina/training_codes/solaris_rina', '/content/DMNet_Rina/training_codes/solaris_rina', '/content/DMNet_Rina/training_codes/solaris_rina', '/usr/local/lib/python3.7/site-packages', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython']
{'weightedbcelogitsrina': None, 'weightedjaccardlossrina': None}


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


=> init weights from normal distribution
!!!!!!!!!!!!!!!!!!!!!! True
=> !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loading pretrained model ../models_imagenet/hrnetv2_w32_imagenet_pretrained.pth
=> loading conv1.weight pretrained model True
=> loading bn1.weight pretrained model True
=> loading bn1.bias pretrained model True
=> loading bn1.running_mean pretrained model True
=> loading bn1.running_var pretrained model True
=> loading bn1.num_batches_tracked pretrained model True
=> loading conv2.weight pretrained model True
=> loading bn2.weight pretrained model True
=> loading bn2.bias pretrained model True
=> loading bn2.running_mean pretrained model True
=> loading bn2.running_var pretrained model True
=> loading bn2.num_batches_tracked pretrained model True
=> loading layer1.0.conv1.weight pretrained model True
=> loading layer1.0.bn1.weight pretrained model True
=> loading layer1.0.bn1.bias pretrained model True
=> loading layer1.0.bn1.running_mean pretrained model True
=> loading layer1.0.bn1

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3722: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


    loss at batch 0: 11.130043983459473
    loss at batch 1: 10.88624382019043
    loss at batch 2: 11.717087745666504
    loss at batch 3: 9.995084762573242
    loss at batch 4: 10.450782775878906
    loss at batch 5: 10.175963401794434
    loss at batch 6: 10.276676177978516
    loss at batch 7: 10.658405303955078
    loss at batch 8: 9.566567420959473
    loss at batch 9: 8.945955276489258
    loss at batch 10: 9.016422271728516
    loss at batch 11: 8.862568855285645
    loss at batch 12: 9.591583251953125
    loss at batch 13: 10.06406307220459
    loss at batch 14: 8.636322975158691
    loss at batch 15: 9.321846961975098
    loss at batch 16: 8.720902442932129
epoch time 6.176530122756958

    Validation loss at epoch 0: 10.44929027557373

Beginning training epoch 1
    loss at batch 0: 9.405881881713867
    loss at batch 1: 9.508959770202637
    loss at batch 2: 8.565550804138184
    loss at batch 3: 7.900786399841309
    loss at batch 4: 9.211902618408203
    loss at batch 5: 

# 4. Run inference

In [9]:
import sys
sys.path.append("./testSet/")
import os 
os.chdir("/content/DMNet_Rina/inference_codes/testSet")
from generate_Detections_78 import get_detection
#from track_bbox import get_noor_track_results as get_noor_track_results_bbox
#from track_mask import get_noor_track_results as get_noor_track_results_mask

import os
import os.path as osp
from post_processing_shape import roi_correct
BF_list=["BF-C2DL-HSC",
            "BF-C2DL-MuSC"]
def main(inp):

    da="BF-C2DL-HSC"
    se="01"
    out_path="BF-C2DL-HSC/01_RES-GT"

    da_name="BF-C2DL-HSC"
    gttype="GT"
    all=int(0)
    ################ generating detection #################
    input_imgpath=osp.join(da,se)
    det_path=out_path
    os.makedirs(det_path, exist_ok=True)
    print (det_path)
    get_detection(input_imgpath, det_path, da_name,gttype,vis=0,all=all)
    #get_noor_track_results(inp)
    ################ generating tracking ##################
    #if da in ['BF-C2DL-MuSC', 'Fluo-C2DL-MSC']:
    #    get_noor_track_results_mask(det_path,da,se,out_path)
    #else:
    #    get_noor_track_results_bbox(det_path,da,se,out_path)

    if da_name in BF_list:
        roi_correct(det_path, det_path)

if __name__ == '__main__':
    in_p=sys.argv
    main(in_p)


ModuleNotFoundError: ignored

# 5. Create a requirements file with the set up of this session

In [ ]:
!pip install ruamel.yaml
import os
os.chdir("/content/DMNet_Rina/training_codes")
import ruamel.yaml

yaml = ruamel.yaml.YAML()
data = yaml.load(open('environment.yml'))

requirements = []
for dep in data['dependencies']:
    if isinstance(dep, str):
        package, package_version, python_version = dep.split('=')
        if python_version == '0':
            continue
        requirements.append(package + '==' + package_version)
    elif isinstance(dep, dict):
        for preq in dep.get('pip', []):
            requirements.append(preq)

with open('requirements.txt', 'w') as fp:
    for requirement in requirements:
       print(requirement, file=fp)